offline editing 10/27/18

todo: 
-1: look into making a data dictionary. need to look up structure and specific function of this. basically i need to annotate what all of my data is, where it comes from, how it's organized (ie by day, by specific time, by icustay, etc) and any limitations of it. 

-2: make a folder in notebooks specifically for older code (specifically for the different clinical variable versions)

-3: refine cohort to only those who have bare minimum vitals. 

-4: questions for nelson and Yuan:

    with missingness table i removed values that were null and counted those as missing, is this kosher? is this case by case basis?
    also need to talk about how I wrangled sofa score and get their opinion.
    

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

final_pt_df2 = pd.read_csv(wd + '/data/raw/csv/16082018_final_pt_df2.csv' , index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 99.3 ms


In [3]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

vaso_dose_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date), index_col=0)
#ventsettings_72=pd.read_csv(
#    wd+'/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date), index_col=0)

ventcategory_df=pd.read_csv(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)

echodata_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date), index_col=0)
labs_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date), index_col=0)


weightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date), index_col=0)
heightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date), index_col=0)

vitals_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date), index_col=0)
uti_all_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date), index_col=0)
bg_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date), index_col=0)

rrt_merged_allpt_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date), index_col=0)
gcs72_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date), index_col=0)

sofa_df_72=pd.read_csv(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14.6 s


In [24]:
def missingness_fxn(df,name, groupby,filteron):
    
    """
    input: dataframe w/ clinical data we want to assess the % missing values on.
    output: a dataframe grouped by the different clinical variables with 
        associated % missingnss (ie how many icustays do not have this value)
        
    """

    missing_df= pd.DataFrame(
        100* (1-(df.groupby(groupby)[filteron].nunique()/ len(hadm_id)))) #number of unique icustays with data /total # icustay
    missing_df.reset_index(inplace=True)
    missing_df= missing_df.rename(index=str, columns={"label":'label', filteron:'%missingness'})
    missing_df['source']= name
    missing_df['data_label']=groupby
    return(missing_df)

time: 7.07 ms


# Goal: to make a new icustay_id cohort with the added criteria that patients have at minimum one Sp02 reading during their first 3 days of suspected infection.
### The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 

In [5]:
vitals_all_nosummary_72.head(2)

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum,t_0,delta
5,9,150750,220597,2149-11-13 15:20:00,MeanArtPress,61.0,2149-11-13,0 days 15:20:00.000000000
6,9,150750,220597,2149-11-14 11:00:00,MeanArtPress,103.0,2149-11-13,1 days 11:00:00.000000000


time: 14.5 ms


In [6]:
#na values
vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values


time: 1.38 s


In [7]:
print(
    vitals_all_nosummary_nonull['icustay_id'].nunique(), #14714/15207 unique icustay_id/icustay_id
)

14714
time: 41 ms


In [9]:
vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2',:]

#missingness_df.loc[missingness_df.loc[:,'label']=='SpO2',:]

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum,t_0,delta
811,9,150750,220597,2149-11-13 15:15:00,SpO2,93.0,2149-11-13,0 days 15:15:00.000000000
812,9,150750,220597,2149-11-14 10:00:00,SpO2,100.0,2149-11-13,1 days 10:00:00.000000000
814,9,150750,220597,2149-11-14 11:00:00,SpO2,100.0,2149-11-13,1 days 11:00:00.000000000
820,9,150750,220597,2149-11-13 06:40:00,SpO2,94.0,2149-11-13,0 days 06:40:00.000000000
821,9,150750,220597,2149-11-14 01:00:00,SpO2,100.0,2149-11-13,1 days 01:00:00.000000000
822,9,150750,220597,2149-11-14 02:00:00,SpO2,99.0,2149-11-13,1 days 02:00:00.000000000
826,9,150750,220597,2149-11-13 09:00:00,SpO2,96.0,2149-11-13,0 days 09:00:00.000000000
830,9,150750,220597,2149-11-13 00:00:00,SpO2,96.0,2149-11-13,0 days 00:00:00.000000000
838,9,150750,220597,2149-11-14 06:00:00,SpO2,100.0,2149-11-13,1 days 06:00:00.000000000
846,9,150750,220597,2149-11-13 17:00:00,SpO2,98.0,2149-11-13,0 days 17:00:00.000000000


time: 520 ms


In [23]:
print((vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].nunique()), #n=14668
    len(icustay_id)
     ) #15207

#print(14668/15207)

icustay_id_vitals = (vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

14668 15207
time: 846 ms


In [28]:

vitals_all_nosummary_nonull_v=(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'icustay_id']
    .isin(icustay_id_vitals),:]
)

14668 15207
time: 1.3 s


In [29]:
#lists and df with _v will be for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
vitals_all_nosummary_nonull_v
icustay_id_v= list(vitals_all_nosummary_nonull_v['icustay_id'].unique())
icustay_id_v= [int(x) for x in icustay_id_v]

time: 51.3 ms


In [32]:
#updated final_pt_df2 for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
final_pt_df2_v = (
    final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_v),:]
)

time: 16.6 ms


In [34]:
final_pt_df2_v.head()

,subject_id,icustay_id,hadm_id,t_0_sc,t_end_sc,ab_course,t_0,t_end_consec,ICU_admit,dod_hosp,dod_ssn,dod_delta,org_list,spec_type_list,sc_result,final_bin
0,24915,267059.0,194424,2182-11-12 01:50:00,2182-11-14 00:00:00,full,2182-11-12,2183-01-15,2182-11-11 20:02:17,2183-01-15 00:00:00,2183-01-15 00:00:00,0 days 00:00:00.000000000,NaN,NaN,negative,C_neg/A_full
1,24942,278729.0,178112,2159-11-07 16:57:00,2159-11-10 00:00:00,partial,2159-11-08,2159-11-10,2159-11-06 11:15:58,NaN,NaN,NaN,NaN,NaN,negative,C_neg/A_partial
2,24968,219670.0,120350,2190-07-31 21:40:00,2190-08-03 00:00:00,partial,2190-08-01,2190-08-02,2190-07-31 20:49:51,2190-08-05 00:00:00,2190-08-05 00:00:00,0 days 00:00:00.000000000,NaN,NaN,negative,C_neg/A_partial
3,24776,295185.0,139951,2154-11-10 23:04:00,2154-11-13 00:00:00,full,2154-11-11,2154-11-19,2154-11-07 20:16:08,2154-11-23 00:00:00,2154-11-23 00:00:00,0 days 00:00:00.000000000,ESCHERICHIA COLI,URINE,positive,C_pos/A_full
4,24714,211472.0,163632,2193-12-11 04:00:00,2193-12-13 00:00:00,full,2193-12-11,2193-12-26,2193-12-11 00:20:56,NaN,NaN,NaN,NaN,NaN,negative,C_neg/A_full


time: 16.6 ms


In [36]:
#todo, crosstab table of pts in new icustay_id_v
final_pt_df2_v.crosstab(final_pt_df2_v['icustay_id'], 
                       columns= final_pt_df2_v['ab_course'],
                       values= final_pt_df2_v['sc_result'])

AttributeError: 'DataFrame' object has no attribute 'crosstab'

time: 253 ms


In [35]:
final_pt_df2.head()

,subject_id,icustay_id,hadm_id,t_0_sc,t_end_sc,ab_course,t_0,t_end_consec,ICU_admit,dod_hosp,dod_ssn,dod_delta,org_list,spec_type_list,sc_result,final_bin
0,24915,267059.0,194424,2182-11-12 01:50:00,2182-11-14 00:00:00,full,2182-11-12,2183-01-15,2182-11-11 20:02:17,2183-01-15 00:00:00,2183-01-15 00:00:00,0 days 00:00:00.000000000,NaN,NaN,negative,C_neg/A_full
1,24942,278729.0,178112,2159-11-07 16:57:00,2159-11-10 00:00:00,partial,2159-11-08,2159-11-10,2159-11-06 11:15:58,NaN,NaN,NaN,NaN,NaN,negative,C_neg/A_partial
2,24968,219670.0,120350,2190-07-31 21:40:00,2190-08-03 00:00:00,partial,2190-08-01,2190-08-02,2190-07-31 20:49:51,2190-08-05 00:00:00,2190-08-05 00:00:00,0 days 00:00:00.000000000,NaN,NaN,negative,C_neg/A_partial
3,24776,295185.0,139951,2154-11-10 23:04:00,2154-11-13 00:00:00,full,2154-11-11,2154-11-19,2154-11-07 20:16:08,2154-11-23 00:00:00,2154-11-23 00:00:00,0 days 00:00:00.000000000,ESCHERICHIA COLI,URINE,positive,C_pos/A_full
4,24714,211472.0,163632,2193-12-11 04:00:00,2193-12-13 00:00:00,full,2193-12-11,2193-12-26,2193-12-11 00:20:56,NaN,NaN,NaN,NaN,NaN,negative,C_neg/A_full


time: 17.8 ms
